In this seminar, you've explored a basic implementation of the Deep Structured Semantic Model (DSSM).

Your task is to **improve this model** in one or more of the following directions:

### ✅ Model Improvements
- [ ] Replace MLP towers with Transformer or RNN encoders or etc. (5 баллов)
- [x] Use different triplet loss. (3 балла)
- [x] Add dropout, batch normalization, or layer norm. (3 балла)
- [x] Integrate embeddings instead of one-hot vectors. (5 баллов)
- [ ] Visualize similarity distribution for positive vs. negative pairs. (5 баллов)

### ✅ Evaluation & Analysis
- [x] Visualize embeddings using t-SNE or UMAP. (3 баллов)
- [x] Develop and improve beyond accuracy metrics. (5 баллов)

### 📄 Deliverables
- [x] Explain what you changed and why in the final markdown cell. (3 балла)
- [x] Keep code modular, clean, and well-documented. (3 балла)

### 📝 Production
- create service based on DSSM vectors with ANN. (8 баллов)

### 📝 Leaderboard
- Improve score from UserKNN via DSSM (8 баллов)


Максимум баллов, которые можно получить - 25.

In [1]:
# ----------------------
# 2. IMPORTS AND SETUP
# ----------------------
import os
import requests
import zipfile
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import warnings
import umap
warnings.filterwarnings("ignore")

/home/evgenii-iurin/work/ITMO-RecSys/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----------------------
# 3. DOWNLOAD AND LOAD DATA
# ----------------------
def download_and_extract():
    url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
    filename = 'kion_train.zip'

    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total = int(response.headers.get('content-length', 0))
        progress = tqdm(response.iter_content(1024 * 1024),
                        f"Downloading {filename}",
                        total=total // (1024 * 1024), unit='MB')
        for chunk in progress:
            f.write(chunk)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("data")
    os.remove(filename)

if not os.path.exists("data/data_original"):
    download_and_extract()



In [3]:
# ----------------------
# 4. DATA PREPROCESSING
# ----------------------
interactions_df = pd.read_csv('data/data_original/interactions.csv', parse_dates=["last_watch_dt"])
users_df = pd.read_csv('data/data_original/users.csv')
items_df = pd.read_csv('data/data_original/items.csv')

# Label Encoding
Вместо использования one-hot векторов закодируем каждую фичу

In [4]:
# Input:
# - user_df - исходный датафрейм с юзерами

# Oupput:
# - user_df - отфильтрованный датафрейм

# Преобразования:
# - Фильтруем не нужные фичи в датафрейме

from sklearn.preprocessing import LabelEncoder

# Заполняем строки "unknown"
for col in ['income', 'sex', 'age']:
    users_df[col] = users_df[col].fillna('unknown')

USERS_FEATURES = ["age", "income", "sex", "kids_flg"]
users_df = users_df[USERS_FEATURES + ["user_id"]]

# Encoding
user_encoders = {}
for feat in USERS_FEATURES:
    le = LabelEncoder()
    users_df[feat] = le.fit_transform(users_df[feat])
    user_encoders[feat] = le

users_df.head()

,age,income,sex,kids_flg,user_id
0,1,4,2,1,973171
1,0,2,2,0,962099
2,3,3,1,0,1047345
3,3,2,1,0,721985
4,2,4,1,0,704055


In [5]:
# Input:
# - items_df - исходный датафрейм с айтемами

# Oupput:
# - items_df - отфильтрованный датафрейм

# Преобразования:
# - Фильтруем не нужные фичи в датафрейме
# - Избавляемся от NaN значений

ITEMS_FEATURES = ['content_type', 'release_year', 'for_kids', 'age_rating', 'studios', 'countries', 'directors']

# Заполняем строки "unknown"
for col in ['content_type', 'studios', 'countries', 'directors']:
    items_df[col] = items_df[col].fillna('unknown')

# Заполняем числовые фичи специальным значением (-1)
for col in ['release_year', 'for_kids', 'age_rating']:
    items_df[col] = items_df[col].fillna(-1)

items_df = items_df[ITEMS_FEATURES + ['item_id']]

item_encoders = {}
for feat in ITEMS_FEATURES:
    le = LabelEncoder()
    items_df[feat] = le.fit_transform(items_df[feat])
    item_encoders[feat] = le

items_df.head()

,content_type,release_year,for_kids,age_rating,studios,countries,directors,item_id
0,0,86,0,4,33,258,5671,10711
1,0,98,0,4,33,421,6546,2508
2,0,95,0,4,33,298,95,10716
3,0,99,0,4,33,57,7735,7868
4,0,62,0,3,34,419,1544,16268


Для каждой фичи пропишем размерность эмбедингов

In [7]:
items_categorical_size = {feat: items_df[feat].nunique() for feat in ITEMS_FEATURES}
items_features_info = [
    (items_categorical_size['content_type'], 8),
    (items_categorical_size['release_year'], 8),
    (items_categorical_size['for_kids'], 2),
    (items_categorical_size['age_rating'], 8),
    (items_categorical_size['studios'], 16),
    (items_categorical_size['countries'], 8),
    (items_categorical_size['directors'], 16),
]

In [8]:
users_categorical_size = {feat: users_df[feat].nunique() for feat in USERS_FEATURES}
users_features_info = [
    (users_categorical_size['age'], 16),
    (users_categorical_size['income'], 16),
    (users_categorical_size['sex'], 2),
    (users_categorical_size['kids_flg'], 2),
]

# Пре-процессинг
В этой секции происходят базовые преобразования, как было показано на лекции. Исключением лишь является то, что фичи пользователей и айтемов не преобразовываются как one-hot вектора, потому что я буду использовать
обучаемые эмбеддинги

In [9]:
# Input:
# - interactions_df — сырые взаимодействия пользователей с фильмами

# Output:
# - Очищенный interactions_df, содержащий:
#   - Только пользователей, просмотревших более 10 фильмов.
#   - Только фильмы, просмотренные более чем 10 пользователями.
#   - Только взаимодействия, где просмотр фильма превышает 10%.

# Преобразования:
# - Удалены взаимодействия с watched_pct ≤ 10%.
# - Оставлены только активные пользователи (с более чем 10 взаимодействиями).
# - Оставлены только популярные фильмы (с более чем 10 пользователями).
# - Итоговый датасет уменьшен в объеме, но очищен для повышения качества обучения модели.


interactions_df = interactions_df[interactions_df.watched_pct > 10]
valid_users = []
c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)
valid_items = []
c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

In [10]:
# Input:
# - interactions_df — очищенные взаимодействия пользователей и фильмов.
# - users_df — таблица с пользователями и их признаками (НЕ one-hot).
# - items_df — таблица с фильмами и их признаками (НЕ one-hot).

# Output:
# - interactions_df, items_df и users_df — синхронизированные таблицы, содержащие только общих пользователей и фильмы.

# Преобразования:
# - Найдены пересечения пользователей и фильмов, присутствующих одновременно в interactions_df и users_df/items_df.
# - Удалены пользователи и фильмы, отсутствующие в обоих соответствующих датасетах.
# - Гарантирована консистентность между взаимодействиями и признаковыми таблицами (users_df и items_df).


common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6901


In [11]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6897


# Пост-процессинг : train и test

### Временной сплит для построения interaction матриц:
- Делим interactions_df по дате на train/test по последним N дней.
- Кодируем user_id и item_id в train через категориальные uid и iid.
- Применяем тот же маппинг к test.
- Строим interaction матрицы отдельно для train и test.
- Таким образом, обучаем модель на истории, а тестируем на предсказании будущих интересов пользователя.


In [12]:
# Разбиваем датасет на трейн и тестовую часть
N_DAYS = 7
max_date = interactions_df['last_watch_dt'].max()

train_df = interactions_df[interactions_df['last_watch_dt'] <= max_date - pd.Timedelta(days=N_DAYS)]
test_df = interactions_df[interactions_df['last_watch_dt'] > max_date - pd.Timedelta(days=N_DAYS)]


# Только юзеры, которые были в train
test_df = test_df[test_df['user_id'].isin(train_df['user_id'])]
# Только айтемы, которые были в train
test_df = test_df[test_df['item_id'].isin(train_df['item_id'])]

In [13]:

# категориально закодируем user_id и item_id по train
train_df['uid'] = train_df['user_id'].astype('category').cat.codes
train_df['iid'] = train_df['item_id'].astype('category').cat.codes

# Сделаем маппинг user_id и item_id в uid и iid

uid_to_user_id = dict(zip(train_df['uid'], train_df['user_id']))
iid_to_item_id = dict(zip(train_df['iid'], train_df['item_id']))

user_id_to_uid = dict(zip(train_df['user_id'], train_df['uid']))
item_id_to_iid = dict(zip(train_df['item_id'], train_df['iid']))

# Создадим колонки uid и iid в test_df и применим маппинг

test_df['uid'] = test_df['user_id'].map(user_id_to_uid)
test_df['iid'] = test_df['item_id'].map(item_id_to_iid)

print(f"Test: {test_df.shape}")
print(f"Train: {train_df.shape}")

Test: (83707, 7)
Train: (1375329, 7)


In [14]:
# Оставим только те айтемы которые есть в train

items_df = items_df[items_df['item_id'].isin(train_df['item_id'])]
items_df = items_df.set_index('item_id').loc[train_df['item_id'].unique()].reset_index()

# Убедимся, что количество уникальных айтомов в items_df совпадает с количеством уникальных айтемов в train_df
assert items_df.item_id.nunique() == train_df.item_id.nunique()

In [15]:
# Оставим только тех юзеров которые есть в train

users_df = users_df[users_df['user_id'].isin(train_df['user_id'])]
users_df = users_df.set_index('user_id').loc[train_df['user_id'].unique()].reset_index()

# Убедимся, что количество уникальных юзеров в users_df совпадает с количеством уникальных юзеров в train_df
assert users_df.user_id.nunique() == train_df.user_id.nunique()

In [16]:
# Проверим, что маппинг был корректный
assert test_df[test_df['uid'] == 4375].user_id.values[0] == train_df[train_df['uid'] == 4375].user_id.values[0]

Сделаем матрицу интеракция для train и test. Размерность одинаковая

In [17]:
import numpy as np

n_users = train_df['uid'].nunique()
n_items = train_df['iid'].nunique()

# ==========================

train_vec = np.zeros((n_users, n_items))
for uid, iid in zip(train_df['uid'], train_df['iid']):
    train_vec[uid, iid] += 1

# нормализация
train_vec = train_vec / train_vec.sum(axis=1, keepdims=True)

# ==========================

# для test
test_vec = np.zeros((n_users, n_items))
for uid, iid in zip(test_df['uid'], test_df['iid']):
    test_vec[uid, iid] += 1

test_vec = test_vec / test_vec.sum(axis=1, keepdims=True)

In [18]:
print(f"Train interaction matrix shape : {train_vec.shape}")
print(f"Test interaction matrix shape : {test_vec.shape}")

Train interaction matrix shape : (65792, 6862)
Test interaction matrix shape : (65792, 6862)


In [20]:
print("Train: Размер уникальных юзеров", train_df.uid.nunique())
print("Test: Размер уникальных юзеров", test_df.uid.nunique())


Train: Размер уникальных юзеров 65792
Test: Размер уникальных юзеров 25676


In [21]:
N_FACTORS = 128

ITEM_MODEL_SHAPE = (items_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (train_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (7,)
USER_META_MODEL_SHAPE: (4,)
USER_INTERACTION_MODEL_SHAPE: (6862,)


# Модель

In [22]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ItemModel(nn.Module):
    def __init__(self, items_features_info, hidden_dim=128, output_dim=128, dropout_rate=0.2):
        """
        Args:
            items_features_info: список кортежей (n_categories, embedding_dim) для каждого признака.
            hidden_dim: размер скрытого слоя.
            output_dim: размер итогового эмбеддинга айтема.
        """
        super(ItemModel, self).__init__()

        # Embedding слои для всех категориальных признаков
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_categories, emb_dim) for num_categories, emb_dim in items_features_info
        ])

        self.emb_total_dim = sum(emb_dim for _, emb_dim in items_features_info)

        # MLP после эмбеддингов
        self.mlp = nn.Sequential(
            nn.Linear(self.emb_total_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, item_features):
        """
        item_features: [batch_size, num_features] (индексы категориальных признаков)
        """
        emb_list = []
        for i, emb_layer in enumerate(self.embeddings):
            emb_list.append(emb_layer(item_features[:, i]))

        x = torch.cat(emb_list, dim=-1)
        x = self.mlp(x)
        return x


In [24]:
class UserModel(nn.Module):
    def __init__(self,
                 categorical_feat_info,
                 interaction_input_dim,
                 hidden_dim=128,
                 output_dim=128,
                 dropout_rate=0.2):
        super(UserModel, self).__init__()

        # Embedding layers для всех категориальных признаков
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_categories, emb_dim) for num_categories, emb_dim in categorical_feat_info
        ])

        self.emb_total_dim = sum(emb_dim for _, emb_dim in categorical_feat_info)

        # MLP для признаков пользователя (meta features)
        self.meta_mlp = nn.Sequential(
            nn.Linear(self.emb_total_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(hidden_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        # MLP для взаимодействий пользователя
        self.interaction_mlp = nn.Sequential(
            nn.Linear(interaction_input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout_rate)
        )

        # Финальный слой после склейки
        self.fc_final = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, meta_features, interaction_features):
        """
        meta_features: [batch_size, num_categorical_features] (индексы признаков)
        interaction_features: [batch_size, interaction_input_dim] (float32 признаки)
        """

        # Обработка мета-фичей через эмбеддинги
        emb_list = []
        for i, emb_layer in enumerate(self.embeddings):
            emb_list.append(emb_layer(meta_features[:, i]))

        meta_embedded = torch.cat(emb_list, dim=-1)

        # Обработка через MLP
        meta_vec = self.meta_mlp(meta_embedded)

        # Обработка взаимодействий
        interaction_vec = self.interaction_mlp(interaction_features)

        # Склейка мета + взаимодействий
        combined = torch.cat([meta_vec, interaction_vec], dim=1)

        # Финальный слой
        user_embedding = self.fc_final(combined)

        return user_embedding


# Loss

In [25]:
import torch.nn.functional as F

def cosine_triplet_loss(anchor, positive, negative, alpha=0.4):
    # Нормализуем вектора
    anchor = F.normalize(anchor, p=2, dim=1)
    positive = F.normalize(positive, p=2, dim=1)
    negative = F.normalize(negative, p=2, dim=1)

    # Косинусное расстояние: 1 - cos_sim
    pos_sim = torch.sum(anchor * positive, dim=1)
    neg_sim = torch.sum(anchor * negative, dim=1)

    basic_loss = neg_sim - pos_sim + alpha
    loss = torch.clamp(basic_loss, min=0.0)
    return loss.mean()

# Dataset

In [26]:
# Define the dataset
class RecSysDataset(Dataset):
    def __init__(self, items, users, interactions, uids: list[int]):
        self.items = items
        self.users = users
        self.interactions = interactions
        self.uids = uids

    def __len__(self):
        return len(self.uids)

    def __getitem__(self, idx):
        uid = self.uids[idx]
        pos_i = np.random.choice(range(self.interactions.shape[1]), p=self.interactions[uid])
        neg_i = np.random.choice(range(self.interactions.shape[1]))
        uid_meta = self.users.iloc[uid].values
        uid_interaction = self.interactions[uid]
        pos = self.items.iloc[pos_i].values
        neg = self.items.iloc[neg_i].values

        return torch.tensor(uid_meta, dtype=torch.int), torch.tensor(uid_interaction, dtype=torch.float32), torch.tensor(pos, dtype=torch.int), torch.tensor(neg, dtype=torch.int), uid

# Инициализация перед тренировкой

In [27]:
i2v = ItemModel(items_features_info)
u2v = UserModel(users_features_info, USER_INTERACTION_MODEL_SHAPE[0])
optimizer = optim.Adam(list(i2v.parameters()) + list(u2v.parameters()), lr=0.001)

dataset = RecSysDataset(items=items_df.drop(["item_id"], axis=1), users=users_df.drop(["user_id"], axis=1), interactions=train_vec, uids=train_df.uid.unique())
test_dataset = RecSysDataset(items=items_df.drop("item_id", axis=1), users=users_df.drop("user_id", axis=1), interactions=test_vec, uids=sorted(test_df.uid.unique()))

dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Accuracy function

In [28]:
def map_at_k(reco_df: pd.DataFrame, ground_truth_df: pd.DataFrame, k: int = 10) -> float:
    """
    Calculate Mean Average Precision at K.
    
    Parameters:
        reco_df: pd.DataFrame with columns [user_id, item_id, rank] — predicted top-K items.
        ground_truth_df: pd.DataFrame with columns [user_id, item_id] — actual relevant items.
        k: cutoff for recommendations.
    
    Returns:
        float: MAP@K score
    """

    # Приведём ground truth в структуру {user_id: set(item_ids)}
    gt_dict = ground_truth_df.groupby("user_id")["item_id"].apply(set).to_dict()

    # Приведём рекомендации в структуру {user_id: [item_ids]}
    reco_df = reco_df[reco_df["rank"] <= k]
    reco_dict = reco_df.sort_values(["user_id", "rank"]).groupby("user_id")["item_id"].apply(list).to_dict()

    average_precisions = []

    for user_id, pred_items in reco_dict.items():
        if user_id not in gt_dict:
            continue  # юзера нет в ground truth

        true_items = gt_dict[user_id]
        if not true_items:
            continue

        num_hits = 0
        score = 0.0

        for i, item in enumerate(pred_items):
            if item in true_items:
                num_hits += 1
                precision_at_i = num_hits / (i + 1)
                score += precision_at_i

        if num_hits > 0:
            ap = score / min(len(true_items), k)
            average_precisions.append(ap)

    return np.mean(average_precisions) if average_precisions else 0.0


In [29]:
import torch
import torch.nn.functional as F
import pandas as pd

def evaluate_map_k(model_user, model_item, test_dataloader, test_df, uid_to_user_id, iid_to_item_id, k=10):
    model_user.eval()
    model_item.eval()

    # Предварительно считаем эмбеддинги всех айтемов
    item_feats = torch.tensor(items_df.drop("item_id", axis=1).values, dtype=torch.int)
    with torch.no_grad():
        item_emb = model_item(item_feats)
        item_emb = F.normalize(item_emb, dim=1)

    reco = []

    with torch.no_grad():
        for batch in test_dataloader:
            uid_meta, uid_interaction, _, _, batch_uids = batch

            user_emb = model_user(uid_meta, uid_interaction)
            user_emb = F.normalize(user_emb, dim=1)

            sim = torch.matmul(user_emb, item_emb.T)  # [batch_size, n_items]
            _, topk_indices = torch.topk(sim, k=k, dim=1)

            for i, item_idxs in enumerate(topk_indices):
                real_uid = batch_uids[i].item()  # uid из датасета
                real_user_id = uid_to_user_id[real_uid]

                for rank, iid in enumerate(item_idxs):
                    reco.append({
                        "user_id": real_user_id,
                        "item_id": iid_to_item_id[iid.item()],
                        "rank": rank + 1
                    })

    reco_df = pd.DataFrame(reco)
    ground_truth = test_df[["user_id", "item_id"]].drop_duplicates()

    score = map_at_k(reco_df, ground_truth, k=k)
    return score


# Training

In [30]:
import logging

# Set up logging
logging.basicConfig(filename='training.log', level=logging.INFO,
                    format='%(asctime)s - %(levelname)s - %(message)s')

losses = []
metrics = []

# Training loop
for epoch in range(2):
    for idx, batch in enumerate(dataloader):
        uid_meta, uid_interaction, pos, neg, _ = batch
        optimizer.zero_grad()
        anchor = u2v(uid_meta, uid_interaction)
        positive = i2v(pos)
        negative = i2v(neg)
        loss = cosine_triplet_loss(anchor, positive, negative)
        loss.backward()
        optimizer.step()

        losses.append(loss.item())
        print(f"Epoch {epoch + 1}, Batch {idx}, Loss: {loss.item()}")
        
        if idx % 100 == 0:
            mapk = map_score = evaluate_map_k(
                        model_user=u2v,
                        model_item=i2v,
                        test_dataloader=test_dataloader,
                        test_df=test_df,
                        uid_to_user_id=uid_to_user_id,
                        iid_to_item_id=iid_to_item_id,
                        k=10
                    )
            metrics.append(mapk)
            print(f"MAP@10: {mapk:.4f}")

    print(f"Epoch {epoch + 1}, Epoch Loss: {loss.item()}")

Epoch 1, Batch 0, Loss: 0.37432432174682617
MAP@10: 0.1187
Epoch 1, Batch 1, Loss: 0.38911646604537964
Epoch 1, Batch 2, Loss: 0.40407219529151917
Epoch 1, Batch 3, Loss: 0.4013963043689728
Epoch 1, Batch 4, Loss: 0.4049313962459564
Epoch 1, Batch 5, Loss: 0.4036625623703003
Epoch 1, Batch 6, Loss: 0.37378084659576416
Epoch 1, Batch 7, Loss: 0.39249545335769653
Epoch 1, Batch 8, Loss: 0.3652392625808716
Epoch 1, Batch 9, Loss: 0.335674911737442
Epoch 1, Batch 10, Loss: 0.37144309282302856
Epoch 1, Batch 11, Loss: 0.40704619884490967
Epoch 1, Batch 12, Loss: 0.36516064405441284
Epoch 1, Batch 13, Loss: 0.40018022060394287
Epoch 1, Batch 14, Loss: 0.4063614308834076
Epoch 1, Batch 15, Loss: 0.3496641516685486
Epoch 1, Batch 16, Loss: 0.30386215448379517
Epoch 1, Batch 17, Loss: 0.37570780515670776
Epoch 1, Batch 18, Loss: 0.38620081543922424
Epoch 1, Batch 19, Loss: 0.3442865312099457
Epoch 1, Batch 20, Loss: 0.40338319540023804
Epoch 1, Batch 21, Loss: 0.35248076915740967
Epoch 1, Batch

Построим график Лосса и метрики

In [ ]:
loss_steps = list(range(1, len(losses) + 1))
metric_steps = list(range(1, len(metrics) + 1))

def moving_average(data, window_size=50):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

losses_smooth = moving_average(losses, window_size=50)
smooth_steps = loss_steps[:len(losses_smooth)]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

ax1.plot(loss_steps, losses, color='red', alpha=0.4, label='Raw Losses')
ax1.plot(smooth_steps, losses_smooth, color='darkred', linewidth=2, label='Smoothed Trend')
ax1.set_title('Losses')
ax1.set_xlabel('Step / Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True)

# График Metrics
ax2.plot(metric_steps, metrics, color='blue')
ax2.set_title('MAP@10')
ax2.set_xlabel('Steps (100 batches)')
ax2.set_ylabel('Metric')
ax2.grid(True)

plt.tight_layout()
plt.show()